# Genetic Algorithm Implementation in Python


这个教程用Python实现了一个简单的遗传算法，我们使用遗传算法求方程式 $y = \sum_{i=1}^6 w_ix_i, -4 < x_i < 4$ 的最大输出。

这个等式有6个输入($x_1$ to $x_6$)和6个权重($w_1$ to $w_6$)，其中权重($w_1, w_2, w_3, w_4, w_5, w_6$) = (4, -2, 3.5, 5, -11, -4.7)， 我们希望找到使该方程最大解。

首先，我们新建一个列表保存权重，一个变量保存解的大小

In [1]:
# The weight of the function.
weights_vector = [4, -2, 3.5, 5, -11, -4.7]
# Number of the weights we are looking to optimize.
x_length = 6

下一步是定义初始群落(population)。根据解的大小，群落中的每个染色体（解）肯定会有6个基因，每条染色体对应一个解，染色体上的每个基因对应解的相对值。

例如一条染色体为`[1,4,2,3,5,6]`，则意味着 $x_1 = 1, x_2 = 4, x_3 = 2, ..., x_6 = 6$

但问题是每个群落中有多少条染色体？没有固定的值，我们可以选择适合我们问题的染色体数量。

接下来，我们创建一个变量，用于保存每个群落的染色体数量，另一个二维`array`用于保存群落，其中群落的大小为 `(染色体数量，染色体长度)`

In [2]:
import numpy as np

chr_per_pop = 8 
# Defining the population size, the population will have sol_per_pop chromosome where each chromosome has num_weights genes.
population_size = (chr_per_pop, x_length)
# Creating the initial population.
new_population = np.random.uniform(low=-4.0, high=4.0, size=population_size)

其中，初始化的群落如下：

In [3]:
new_population

array([[-1.07096355,  2.12637903, -3.65055675,  0.89392587,  0.31945297,
        -1.78857501],
       [-3.10042758,  0.4585576 ,  0.32404894,  3.95106426, -0.80153147,
        -3.20079391],
       [ 0.83449388, -2.14438494, -3.79974769,  3.83373239, -0.34800213,
        -3.40924413],
       [ 2.17018714, -2.92698311, -3.50872556, -1.99956324,  0.18518237,
         1.36187346],
       [ 2.985333  , -2.68037602,  1.68890233, -1.78531959, -2.13205835,
        -3.19741739],
       [-1.94783842,  1.80685784,  3.32590962,  0.582248  ,  2.19164785,
        -1.87387736],
       [-1.32454395, -1.85216013, -1.4060029 , -2.36212197, -2.00566806,
        -1.18126957],
       [-0.46917962,  3.78800549,  3.20883708, -1.13345093,  0.59462602,
         3.91858743]])

准备好群落（第一代群落）后，接下来根据适应度函数，我们将选择当前群落中最好的个体作为交配亲本，也称为精英染色体。

下一步是应用遗传变异（交叉和变异），根据精英染色体来产生下一代的后代染色体。精英染色体和遗传变异得到的下一代染色体组成新的群体（下一代群落），并在多次迭代/遗传中重复这些步骤

首先，我们定义迭代次数`num_generations`，即遗传多少代后停止。然后定义每一代中精英染色体数量`num_parents_mating`，即选出作为交配亲本的染色体数量

In [4]:
num_generations = 5
num_parents_mating = 4 # the num of parents selected by population in each generation

现在我们已经拥有了初始化的群落`new_population`和方程的权重向量`weight_vector`，第一步要计算群落中每条染色体的适应度（即方程的解大小）。

因此我们需要一个计算适应度的函数`col_pop_fitness(weight_vector, population)`，输入群落和权重向量，得到群落中每条染色体的适应度

In [5]:
def col_pop_fitness(weights_vector, pop):
    # Calculating the fitness value of each chromosome in the current population.
    # The fitness function caulcuates the sum of products between each input and its corresponding weight.
    fitness = np.sum(pop*weights_vector, axis=1)
    return fitness

接下来，我们从群落中选择适应度最大的`n`条染色体，做为精英染色体，用于交配下一代。为此我们需要一个选择精英染色体的函数

`select_mating_pool(population, fitness, num_parents_mating)`

输入群落，适应度和精英染色体数量，得到精英染色体

In [6]:
def select_mating_pool(pop, fitness, num_parents):
    # Selecting the best chromosome in the current generation as parents for producting the offspring of the next generation.
    parents = np.empty((num_parents, pop.shape[1]))
    for parent_num in range(num_parents):
        max_fitness_idx = np.where(fitness == np.max(fitness))
        max_fitness_idx = max_fitness_idx[0][0]
        parents[parent_num, :] = pop[max_fitness_idx, :]
        fitness[max_fitness_idx] = -float('inf')
    return parents

接下来，我们对精英染色体进行交配得到下一代。基因交换如图

![](mating.png)

In [7]:
def crossover(parents, offspring_size):
    offspring = np.empty(offspring_size)
    # The point at which crossover takes place between two parents. Usually, it is at center
    crossover_point = np.int64(offspring_size[1]/2)

    for k in range(offspring_size[0]):
        # Index of the first parent to mate.
        parent1_idx = np.random.randint(0, parents.shape[0])
        # Index of the second parent to mate
        parent2_idx = np.random.randint(0, parents.shape[0])
        # The new offspring will have its genes from parents
        offspring[k, :] = np.array([*parents[parent1_idx, :crossover_point], *parents[parent2_idx, crossover_point:]])
    return offspring

下一步，为了提高算法的泛化能力，我们对得到的下一代染色体进行基因变异，即随机选择染色体的一个基因，将其变成一个随机值

输入群落，得到变异后的群落

In [8]:
def mutation(offspring_population):
    # Mutation changes a single gene in each offspring randomly.
    for idx in range(offspring_population.shape[0]):
        # The random value to replace the gene.
        random_value = np.random.uniform(-4.0, 4.0, 1)
        offspring_population[idx, np.random.randint(0,offspring_population.shape[1])] = random_value
    return offspring_population

对上述过程重复迭代`num_generations`次，就得到了最终的结果

In [9]:
for generation in range(num_generations):
    print("Generation : ", generation)

    # Calculationg the fitness of each chromosome in the population.
    fitness = col_pop_fitness(weights_vector, new_population)

    # Selecting the best parents in the population for mating.
    parents = select_mating_pool(new_population, fitness, num_parents_mating)

    # Generating next generation using crossover.
    offspring_crossover = crossover(parents, offspring_size=(population_size[0]-parents.shape[0], x_length))

    # Change some genes in offspring using mutation.
    offspring_mutation = mutation(offspring_crossover)

    # Creating the new population based on the parents and offspring.
    new_population[0:parents.shape[0], :] = parents
    new_population[parents.shape[0]:, :] = offspring_mutation

    # The best result in the current iteration.
    print("Best result : ", np.max(np.sum(new_population*weights_vector, axis=1)))

# Getting the best solution after iterating finishing all generation
# At first, the fitness is calculated for each solution in the final generation
fitness = col_pop_fitness(weights_vector, new_population)

# Then return the index of the solution corresponding the best fitness
best_match_idx = np.where(fitness == np.max(fitness))

print("Best solution : ", new_population[best_match_idx, :])
print("Best solution fitness : ", fitness[best_match_idx])


Generation :  0
Best result :  52.767147926086125
Generation :  1
Best result :  70.0439800822489
Generation :  2
Best result :  70.0439800822489
Generation :  3
Best result :  70.0439800822489
Generation :  4
Best result :  71.36709917998454
Best solution :  [[[ 2.92056594 -2.68037602  3.68789099  3.83373239 -1.1501908
   -2.04163919]]]
Best solution fitness :  [71.36709918]


最后，汇总的代码如下：

In [4]:
import numpy as np

# The weight of the function.
weights_vector = [4, -2, 3.5, 5, -11, -4.7]
# Number of the weights we are looking to optimize.
x_length = 6
# Number of chromosome in a population
chr_per_pop = 8 
# Defining the population size, the population will have sol_per_pop chromosome where each chromosome has num_weights genes.
population_size = (chr_per_pop, x_length)
# Creating the initial population.
new_population = np.random.uniform(low=-4.0, high=4.0, size=population_size)
# Number of iterations
num_generations = 100
# The num of parents selected by population in each generation
num_parents_mating = 4 


def col_pop_fitness(equation_inputs, pop):
    # Calculating the fitness value of each solution in the current population.
    # The fitness function caulcuates the sum of products between each input and its corresponding weight.
    fitness = np.sum(pop*equation_inputs, axis=1)
    return fitness

def select_mating_pool(pop, fitness, num_parents):
    # Selecting the best chromosome in the current generation as parents for producting the offspring of the next generation.
    parents = np.empty((num_parents, pop.shape[1]))
    for parent_num in range(num_parents):
        max_fitness_idx = np.where(fitness == np.max(fitness))
        max_fitness_idx = max_fitness_idx[0][0]
        parents[parent_num, :] = pop[max_fitness_idx, :]
        fitness[max_fitness_idx] = -float('inf')
    return parents

def crossover(parents, offspring_size):
    offspring = np.empty(offspring_size)
    # The point at which crossover takes place between two parents. Usually, it is at center
    crossover_point = np.int64(offspring_size[1]/2)

    for k in range(offspring_size[0]):
        # Index of the first parent to mate.
        parent1_idx = np.random.randint(0, parents.shape[0])
        # Index of the second parent to mate
        parent2_idx = np.random.randint(0, parents.shape[0])
        # The new offspring will have its genes from parents
        offspring[k, :] = np.array([*parents[parent1_idx, :crossover_point], *parents[parent2_idx, crossover_point:]])
    return offspring

def mutation(offspring_crossover):
    # Mutation changes a single gene in each offspring randomly.
    for idx in range(offspring_crossover.shape[0]):
        # The random value to be added to the gene.
        random_value = np.random.uniform(-4.0, 4.0, 1)
        offspring_crossover[idx, np.random.randint(0,offspring_crossover.shape[1])] = random_value
    return offspring_crossover


def main(show_detail):
    for generation in range(num_generations):
        print("Generation : ", generation)

        # Calculationg the fitness of each chromosome in the population.
        fitness = col_pop_fitness(weights_vector, new_population)
        if show_detail: print("Fitness Value : \n", fitness)

        # Selecting the best parents in the population for mating.
        parents = select_mating_pool(new_population, fitness, num_parents_mating)
        if show_detail: print("Selected parents : \n", parents)

        # Generating next generation using crossover.
        offspring_crossover = crossover(parents, offspring_size=(population_size[0]-parents.shape[0], x_length))
        if show_detail: print("Crossover result : \n", offspring_crossover)

        # Change some genes in offspring using mutation.
        offspring_mutation = mutation(offspring_crossover)
        if show_detail: print("Mutation result : \n", offspring_mutation)

        # Creating the new population based on the parents and offspring.
        new_population[0:parents.shape[0], :] = parents
        new_population[parents.shape[0]:, :] = offspring_mutation

        # The best result in the current iteration.
        print("Best result : ", np.max(np.sum(new_population*weights_vector, axis=1)))

    # Getting the best solution after iterating finishing all generation
    # At first, the fitness is calculated for each solution in the final generation
    fitness = col_pop_fitness(weights_vector, new_population)

    # Then return the index of the solution corresponding the best fitness
    best_match_idx = np.where(fitness == np.max(fitness))

    print("Best solution : ", new_population[best_match_idx, :])
    print("Best solution fitness : ", fitness[best_match_idx])


if __name__ == '__main__':
    main(False)

Generation :  0
Best result :  78.85749097469876
Generation :  1
Best result :  85.90456955728268
Generation :  2
Best result :  85.90456955728268
Generation :  3
Best result :  85.90456955728268
Generation :  4
Best result :  108.21473509937029
Generation :  5
Best result :  108.21473509937029
Generation :  6
Best result :  108.21473509937029
Generation :  7
Best result :  108.21473509937029
Generation :  8
Best result :  108.21473509937029
Generation :  9
Best result :  108.21473509937029
Generation :  10
Best result :  108.21473509937029
Generation :  11
Best result :  108.3619923784129
Generation :  12
Best result :  108.3619923784129
Generation :  13
Best result :  108.3619923784129
Generation :  14
Best result :  108.3619923784129
Generation :  15
Best result :  109.04917015468945
Generation :  16
Best result :  109.04917015468945
Generation :  17
Best result :  109.04917015468945
Generation :  18
Best result :  109.04917015468945
Generation :  19
Best result :  109.0491701546894